# Melero Approach

The task to be solved with this Notebook is the replication of the "Melero Approach". First, all possible patent applications of researchers as inventors are researched. Therefore, assignees and assignors are filtered by university.

The approach is based on 3 different data sets: PatEx, Patentsview & Patent Assignment.


In [1]:
# Basic import
import pandas as pd
import csv
import numpy as np
import requests
import os


In [2]:
# Extend display
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
# Set the paths correctly
cwd_path = os.getcwd()
raw_data_path = os.path.join(cwd_path, "raw_data").replace("\\","/")


In [4]:
raw_data_path

'C:/Users/Lucas/code/prinzpoker/notebooks/raw_data'

# Patent application number 
Only universities

## PatEx - USPTO
PatEx data are generally representative of the population of patent applications filed in the United States after November 2000 across observable characteristics.

Different Data sets (we concentrate on three: application_data; all_inventors; status_codes)

### application_data

In [5]:
df_application = pd.DataFrame()
df_application = pd.read_csv(os.path.join(raw_data_path,"uspto/application_data.csv").replace("\\","/"), delimiter = ",")

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (0,1,3,4,5,6,8,9,10,11,12,13,14,16,17,18,19,20,21) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [6]:
df_application.head()

,application_number,filing_date,application_invention_type,examiner_full_name,examiner_art_unit,uspc_class,uspc_subclass,confirm_number,atty_docket_number,appl_status_desc,appl_status_date,file_location,file_location_date,earliest_pgpub_number,earliest_pgpub_date,wipo_pub_number,wipo_pub_date,patent_number,patent_issue_date,invention_title,small_entity_indicator,aia_first_to_file
0,04453098,NaN,NaN,"LATEEF, MARVIN M",2106,338,254000,6933.0,NaN,Patented File - (Old Case Added for File Track...,1983-12-28,FILE REPOSITORY (FRANCONIA),1986-04-23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDISCOUNTED,NaN
1,04544040,NaN,NaN,"GLUCK, RICHARD E",3403,417,353000,9431.0,NaN,Patented File - (Old Case Added for File Track...,1983-08-22,FILE REPOSITORY (FRANCONIA),1999-06-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDISCOUNTED,NaN
2,04577552,NaN,NaN,"LOOK, EDWARD K",3401,060,053000,1435.0,NaN,Patented File - (Old Case Added for File Track...,1984-02-21,FILE REPOSITORY (FRANCONIA),2008-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDISCOUNTED,NaN
3,04578318,NaN,NaN,"HOWELL, DANIEL W",3722,409,163000,1455.0,NaN,ABANDONED - RESTORED,1988-01-28,FILE REPOSITORY (FRANCONIA),2016-11-21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,UNDISCOUNTED,NaN
4,04589018,NaN,Utility,"NOT, DEFINED",2401,220,271000,1808.0,NaN,Patented Case,1980-01-15,FILE REPOSITORY (FRANCONIA),1995-09-06,NaN,NaN,NaN,NaN,4183442,1980-01-15,NaN,UNDISCOUNTED,NaN


In [6]:
df_application.shape

(16514638, 22)

'>16 Mio. -> Reducing data: appl_status_date: 2000-01-01 bis 2016-01-01
 & only type "Utility" (see Melero)


In [7]:
df2_application = pd.DataFrame()
df2_application = df_application.loc[(df_application["filing_date"] > '2000-12-31') & (df_application["filing_date"] <= '2015-12-31'),:]
df2_application = df2_application.loc[df2_application["application_invention_type"]=="Utility",:]
df2_application["application_number"] = df2_application["application_number"].astype("string")

In [8]:
df2_application.shape

(4938053, 22)

Which columns are useless? -> "invention_title"; "small_entity_indicator" ; "wipo_pub_date"; "wipo_pub_number" 

In [9]:
df2_application.drop(columns = {"invention_title", "small_entity_indicator", "wipo_pub_date", "wipo_pub_number"}, inplace = True)

## Patentsview
First data set that includes information about assignees


### application 
preGrant-Application

In [10]:
df3_application = pd.DataFrame()
df3_application = pd.read_csv(os.path.join(raw_data_path,"patentsview/application.tsv").replace("\\","/"), delimiter = "\t")

In [11]:
df3_application.head()

,id,document_number,type,application_number,date,country,series_code,invention_title,invention_abstract,rule_47_flag,filename
0,0000021c-ccf9-11ea-ba95-121df0c29c1e,20100070354,utility,12412361,2009-03-27,US,12,SYSTEM AND METHOD FOR A MERCHANT DEBIT CARD PR...,A merchant debit card program is described tha...,False,ipa100318.xml
1,00001b62-f3c4-11eb-b0cf-121df0c29c1e,20210172812,utility,16623118,2018-04-20,US,16,"ENERGY CONVERTER, VIBRATION POWER GENERATOR, F...",An energy converter is formed by bonding a sol...,False,NaN
2,00004fba-ccf9-11ea-ba95-121df0c29c1e,20100070355,utility,12623189,2009-11-20,US,12,Methods for Transmitting Multimedia Files and ...,The invention is directed to a method of trans...,False,ipa100318.xml
3,0000525a-ccf9-11ea-ba95-121df0c29c1e,20100070356,utility,12516762,2007-09-19,US,12,METHOD AND SYSTEM OF SALES PROMOTION,A method of sales promotion using a networked ...,False,ipa100318.xml
4,00007a2d-ccf9-11ea-ba95-121df0c29c1e,20100070357,utility,12334277,2008-12-12,US,12,INCENTIVE BASED MARKETING THROUGH SOCIAL NETWORKS,A method and system for providing an incentive...,False,ipa100318.xml


In [12]:
df3_application["application_number"] = df3_application["application_number"].astype("string")


### assignee & publication_assignee

In [13]:
df_assignee = pd.DataFrame()
df_assignee = pd.read_csv(os.path.join(raw_data_path,"patentsview/assignee.tsv").replace("\\","/"),delimiter="\t") 

In [14]:
df_assignee.head()

,id,type,name_first,name_last,organization
0,00002ded-cef9-4c06-ad0c-0fee8891a8ed,2.0,NaN,NaN,"Butterick Company, Inc."
1,00002ed6-a81c-4adf-afa3-e91961107dca,3.0,NaN,NaN,Conros Corporation
2,000055d3-0d65-4d07-8d0a-8939b578b0e1,3.0,NaN,NaN,Chungbuk National University
3,0000591f-7548-49ee-a4ae-fca3b0c10b1c,3.0,NaN,NaN,TELEVIC CONFERENCE NV
4,00007585-cd5c-46d6-96ea-09042748a550,3.0,NaN,NaN,ACES INGENIEURGESELLSCHAFT MBH


Reduce assignees -> Only University

In [15]:
df2_assignee = pd.DataFrame()
df2_assignee = df_assignee.loc[(df_assignee["organization"].str.contains("University") ==True) |(df_assignee["organization"].str.contains("UNIVERSITY") ==True)|(df_assignee["organization"].str.contains("Faculty") ==True) | (df_assignee["name_first"].str.contains("FACULTY")==True) | (df_assignee["name_first"].str.contains("INSTITUTE")==True) | (df_assignee["name_last"].str.contains("Institute")==True),:]

In [16]:
df2_assignee.head()

,id,type,name_first,name_last,organization
2,000055d3-0d65-4d07-8d0a-8939b578b0e1,3.0,NaN,NaN,Chungbuk National University
109,000f1730-c563-4dcf-b14e-257093c523fb,NaN,NaN,NaN,The Ohiio State University Research Foundation
221,001b46be-be17-429c-9ed9-5e07acc0e2ad,3.0,NaN,NaN,ZHONGKAI UNIVERSITY OF AGRICULTURE AND ENGINEER
263,0020f5ec-7144-4a26-949b-f89e1b86872e,2.0,NaN,NaN,University of Tennesseee Research Foundation
288,0023bcd6-ffbc-4423-9507-32b97f9d88c1,3.0,NaN,NaN,University of Shanghai for Science and Technology


In [17]:
df2_assignee.shape

(4825, 5)

In [19]:
df_publication_assignee = pd.DataFrame()
df_publication_assignee = pd.read_csv(os.path.join(raw_data_path,"patentsview/publication_assignee.tsv").replace("\\","/"), delimiter="\t")

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [20]:
df_publication_assignee.head()

,document_number,assignee_id,sequence,location_id
0,20050000014,2da62f97-01de-4da0-a945-87f4ebf0dd6a,1,f54d6149-cb8e-11eb-9615-121df0c29c1e
1,20050000023,03aa96b7-46eb-4f8e-b90e-906cdbcc08bb,1,cfe77bc3-cb8e-11eb-9615-121df0c29c1e
2,20050000025,0982f119-65f7-4662-802f-74f5d28d6082,1,d6338035-cb8e-11eb-9615-121df0c29c1e
3,20050000029,a9777969-6389-4729-9045-949ce0b2006b,1,3cb80671-cb8e-11eb-9615-121df0c29c1e
4,20050000031,a9777969-6389-4729-9045-949ce0b2006b,1,3cb80671-cb8e-11eb-9615-121df0c29c1e


### Merge Patentsview
with assignee=ONLY UNIVERSITY

In [21]:
df_patentsview = pd.DataFrame()
df_patentsview = pd.merge(df3_application, df_publication_assignee, left_on ="document_number",right_on = "document_number")

In [22]:
df_patentsview.head()

,id,document_number,type,application_number,date,country,series_code,invention_title,invention_abstract,rule_47_flag,filename,assignee_id,sequence,location_id
0,0000021c-ccf9-11ea-ba95-121df0c29c1e,20100070354,utility,12412361,2009-03-27,US,12,SYSTEM AND METHOD FOR A MERCHANT DEBIT CARD PR...,A merchant debit card program is described tha...,False,ipa100318.xml,8627328a-b3ad-41d0-9a65-292da36aa366,1,f9139cb2-cb8f-11eb-9615-121df0c29c1e
1,00001b62-f3c4-11eb-b0cf-121df0c29c1e,20210172812,utility,16623118,2018-04-20,US,16,"ENERGY CONVERTER, VIBRATION POWER GENERATOR, F...",An energy converter is formed by bonding a sol...,False,NaN,68360f1f-9ba5-4f86-8463-5fd48de70a81,1,NaN
2,00001b62-f3c4-11eb-b0cf-121df0c29c1e,20210172812,utility,16623118,2018-04-20,US,16,"ENERGY CONVERTER, VIBRATION POWER GENERATOR, F...",An energy converter is formed by bonding a sol...,False,NaN,b0282278-f579-405a-a37e-6dca490b7b86,2,NaN
3,00004fba-ccf9-11ea-ba95-121df0c29c1e,20100070355,utility,12623189,2009-11-20,US,12,Methods for Transmitting Multimedia Files and ...,The invention is directed to a method of trans...,False,ipa100318.xml,5ec9b9e8-545a-4505-bde2-a04fb02c9d07,1,fd10d2b1-cb8e-11eb-9615-121df0c29c1e
4,00007a2d-ccf9-11ea-ba95-121df0c29c1e,20100070357,utility,12334277,2008-12-12,US,12,INCENTIVE BASED MARKETING THROUGH SOCIAL NETWORKS,A method and system for providing an incentive...,False,ipa100318.xml,b0e41602-d133-4b02-9566-e03b1fdb55ae,1,ffc9f8bd-cb8e-11eb-9615-121df0c29c1e


In [23]:
df_patentsview = pd.merge(df_patentsview,df2_assignee, left_on = "assignee_id", right_on = "id")

In [24]:
# Reduce only application_number left
df_appno_patentsview = pd.DataFrame()
df_appno_patentsview["application_number"] = df_patentsview["application_number"]

In [25]:
df_appno_patentsview.shape

(95324, 1)

In [26]:
df_appno_patentsview.head()

,application_number
0,16623118
1,12589849
2,12589852
3,12589949
4,14342829


## Patent Assignment
Second data set that includes information about assignees (and assignors)

### documentid_admin

In [27]:
df_document = pd.DataFrame()
df_document = pd.read_csv(os.path.join(raw_data_path,"uspto/documentid_admin.csv").replace("\\","/"), delimiter=",")

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (1,2,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
df_document.head()

,rf_id,appno_doc_num,grant_doc_num,admin_appl_id_for_grant,admin_pat_no_for_appno,error
0,53050866,NaN,3844976,NaN,NaN,missing appno_doc_num
1,206900830,NaN,0866957,NaN,NaN,missing appno_doc_num
2,58470541,NaN,0561661,NaN,NaN,missing appno_doc_num
3,57910252,NaN,4164286,NaN,NaN,missing appno_doc_num
4,52700026,NaN,4030173,NaN,NaN,missing appno_doc_num


In [29]:
### Clean dataframe df_document -> preperation for merge by "appno_doc_num"

df_document =df_document.loc[df_document.appno_doc_num.isna()==False, :]
df_document = df_document.loc[df_document.error != "incorrect appno_doc_num",:]
df_document.appno_doc_num = df_document.appno_doc_num.astype("string")
df_document.appno_doc_num= df_document.appno_doc_num.str.replace(".","")

<ipython-input-29-043d6e481dd1>:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_document.appno_doc_num= df_document.appno_doc_num.str.replace(".","")


In [30]:
df_document.shape

(15005371, 6)

### assignee

In [31]:
df3_assignee = pd.DataFrame()
df3_assignee = pd.read_csv(os.path.join(raw_data_path,"uspto/assignee.csv").replace("\\","/"),delimiter=",")

In [32]:
df3_assignee.head()

,rf_id,ee_name,ee_address_1,ee_address_2,ee_city,ee_state,ee_postcode,ee_country
0,12800340,CHILDREN'S MEDICAL CENTER CORPORATION,"55 SHATTUCK STREET BOSTON, MA 02115",NaN,NaN,NaN,NaN,NaN
1,36250888,NORTHERN TELECOM LIMITED,NaN,NaN,NaN,NaN,NaN,NOT PROVIDED
2,36340874,PARADYNE CORPORATION,NaN,8550 ULMERTON ROAD,LARGO,FLORIDA,33540,NaN
3,36340875,PARADYNE CORPORATION,NaN,8550 ULMERTON ROAD,LARGO,FLORIDA,33540,NaN
4,36920537,"EHD, INC.",A TENNESSEE CORPORATION,500 GOULD DRIVE,COOKEVILLE,TENNESSEE,38501,NaN


In [33]:
### Selection of "university"
df4_assignee = pd.DataFrame()
df4_assignee = df3_assignee.loc[(df3_assignee["ee_name"].str.contains("University")==True) | (df3_assignee["ee_name"].str.contains("UNIVERSITY")==True)| (df3_assignee["ee_name"].str.contains("Institute")==True)| (df3_assignee["ee_name"].str.contains("INSTITUTE")==True)| (df3_assignee["ee_name"].str.contains("Faculty")==True)| (df3_assignee["ee_name"].str.contains("FACULTY")==True), :]

In [34]:
df4_assignee.shape

(394209, 8)

#### Merge assignee (only university)

In [35]:
df4_assignee = pd.merge(df4_assignee, df_document, on="rf_id")

In [37]:
# Reduce for application number 
df_app_no_assignee = pd.DataFrame()
df_app_no_assignee["application_number"] = df4_assignee["appno_doc_num"]

In [39]:
df_app_no_assignee.head()

,application_number
0,5935638
1,5729465
2,6128800
3,6162524
4,6086647


In [40]:
df_app_no_assignee.shape

(410740, 1)

### assignor

In [41]:
df_assignor = pd.DataFrame()
df_assignor = pd.read_csv(os.path.join(raw_data_path,"uspto/assignor.csv").replace("\\","/"), delimiter=",")

c:\users\lucas\appdata\local\programs\python\python38\lib\site-packages\IPython\core\interactiveshell.py:3331: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
df_assignor.shape

(21254300, 4)

In [43]:
df_assignor.head()

,rf_id,or_name,exec_dt,ack_dt
0,12800340,"ATALA, ANTHONY",1994-12-22,NaN
1,36250888,NORTHERN ELECTRIC COMPANY LIMITED,1976-03-01,NaN
2,36340874,"ARMSTRONG, THOMAS R.",1979-02-23,NaN
3,36340875,"ARMSTRONG, THOMAS R.",1979-02-23,NaN
4,36920537,GOULD INC.,1978-10-31,NaN


In [44]:
# Changed assignee: after year 2000
df_assignor = df_assignor.loc[(df_assignor["exec_dt"] > '2000-12-31') & (df_assignor["exec_dt"] <= '2015-12-31'),:]

In [45]:
df_assignor.shape

(12872859, 4)

In [46]:
### Selection of "university" 
df2_assignor = pd.DataFrame()
df2_assignor = df_assignor.loc[(df_assignor["or_name"].str.contains("UNIVERSITY")==True) | (df_assignor["or_name"].str.contains("University")==True)| (df_assignor["or_name"].str.contains("Institute")==True)| (df_assignor["or_name"].str.contains("INSTITUTE")==True)| (df_assignor["or_name"].str.contains("Faculty")==True)| (df_assignor["or_name"].str.contains("FACULTY")==True),:]

In [47]:
df2_assignor = pd.merge(df2_assignor,df_document,on="rf_id")

In [48]:
df2_assignor_application = pd.DataFrame()
df2_assignor_application["application_number"]= df2_assignor["appno_doc_num"]

In [49]:
df2_assignor_application.head()

,application_number
0,9562328
1,7761407
2,9270737
3,9617033
4,9677972


### Merge of all 3 data set 
according assignee=university

In [50]:
#Join all application numbers
df_applications=pd.DataFrame()
df_applications=pd.concat([df2_assignor_application,df_app_no_assignee,df_appno_patentsview])


In [51]:
# Eliminate duplicates -> Final data set for application numbers
df_applications.drop_duplicates(subset=["application_number"], inplace = True)

In [52]:
df_applications.shape

(341027, 1)

In [56]:
# Upload as csv final_data
df_applications.to_csv("raw_data/application_numbers.csv", index=False)

## Next Steps
#### selected patent applications merge with applications(PateEx)
--------------------------------------------------------------
#### use selected patent publication numbers in Dimensions

In [57]:
df_numbers = pd.DataFrame()
df_numbers = pd.read_csv(os.path.join(raw_data_path,"application_numbers.csv").replace("\\","/"), delimiter=",")

In [58]:
df_numbers.head()

,application_number
0,9562328
1,7761407
2,9270737
3,9617033
4,9677972


In [60]:
df_dimensions = pd.DataFrame()
df_dimensions = pd.merge(df2_application,df_numbers, on="application_number")

In [61]:
df_dimensions.head()

,application_number,filing_date,application_invention_type,examiner_full_name,examiner_art_unit,uspc_class,uspc_subclass,confirm_number,atty_docket_number,appl_status_desc,appl_status_date,file_location,file_location_date,earliest_pgpub_number,earliest_pgpub_date,patent_number,patent_issue_date,aia_first_to_file
0,10227652,2001-03-28,Utility,"MARC, MCDIEUNEL",3661,700,245000,4659.0,CIT 3189,Patented Case,2004-02-05,FILE REPOSITORY (FRANCONIA),2018-06-22,US20030074108A1,2003-04-17,6697710,2004-02-24,false
1,10041727,2001-04-12,Utility,"HUGHES, DEANDRA M",3663,359,349000,4488.0,06666-075001/USC-2966,Patent Expired Due to NonPayment of Maintenanc...,2015-10-12,FILE REPOSITORY (FRANCONIA),2018-06-23,US20020167722A1,2002-11-14,6621627,2003-09-16,false
2,10294044,2001-07-18,Utility,"NGUYEN, TANH Q",2182,710,100000,8124.0,FULCP014C2,Abandoned -- Failure to Respond to an Office A...,2007-08-31,ELECTRONIC,NaN,US20040030858A1,2004-02-12,NaN,NaN,false
3,10076918,2001-10-11,Utility,"SWOPE, SHERIDAN",1652,435,023000,8460.0,UTSD:857US/SLH,Abandoned -- Failure to Respond to an Office A...,2005-06-03,ELECTRONIC,NaN,US20030073097A1,2003-04-17,NaN,NaN,false
4,10004348,2001-10-18,Utility,"LY, CHEYNE D",2168,702,019000,3700.0,STAN-182,Abandoned -- Failure to Respond to an Office A...,2005-10-20,ELECTRONIC,NaN,US20020146371A1,2002-10-10,NaN,NaN,false


In [65]:
# earliest_pgpub_number => publication number for dimensions
df_dimensions= df_dimensions.loc[df_dimensions["earliest_pgpub_number"].isna()==False,:]

In [66]:
### 191.953(1) different publication numbers ###
df_dimensions.shape

(191953, 18)

In [68]:
# csv upload
df_dimensions["earliest_pgpub_number"].to_csv("raw_data/publication_numbers.csv", index=False)